In [19]:
import cv2
import numpy as np

In [20]:
def region_of_interest(img, vertices):
    # Define a blank matrix that matches the image height/width.
    mask = np.zeros_like(img)
    
    # Retrieve the number of color channels of the image.
    #channel_count = img.shape[2]
    
    #check if image is grayscale or color
    shape = img.shape
    
    # color used to fill polygon
    if(len(shape) == 3):               #its a color image
        mask_color = (255,)*shape[-1]   #shape[-1] = no. of channels
    else:                              #its a gray image or single channel
        mask_color = 255
      
    # Fill the polygon with white
    cv2.fillPoly(mask, vertices, mask_color)
    
    # Returning the image only where mask pixels match
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

In [21]:
#Applying canny edge and thresholding using median of pixels
def autoCanny(img, sigma=0.33):
    img = img[:,:,2]
    #get median of gray scale image
    med = np.median(img)
    
    #compute lower and upper threshold
    lower = int(max(0, (1-sigma)*med))
    upper = int(min(255, (1+sigma)*med))
    edgedImg = cv2.Canny(image, lower, upper)
    
    #return image with prominent edges
    return edgedImg

In [ ]:
#Convert hsv to gray scale
def hsv2gray(img):
    #initialize gray img
    h, w, c = img.shape
    gray = hsv[:,:,2]
    return gray

In [22]:
cap = cv2.VideoCapture('../dataset/night-lane.mp4')

#get height and width of frame
ret, frame = cap.read()
height, width, channel = frame.shape

#keeping the portion of image which contains lane
topLeftPt = (width*(3.0/8), height*(3/5))
topRightPt = (width*(5.0/8), height*(3/5))

#polygon has 6 vertices
region_of_interest_points = [
(0, height),
(0, height*(3.4/5)),
topLeftPt,
topRightPt,
(width, height*(3.4/5)),
(width, height),
]

while True:

    # Take each frame
    ret, frame = cap.read()
    
    #convert it to gray scale
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    cv2.imshow('grayImg', gray)
    
    #perform canny edge on gray scale image
    cannyImg = autoCanny(gray, 0.33)
    #cv2.imshow('cannyImg', cannyImg)
    
    #crop image
    croppedImg = region_of_interest(cannyImg, np.array([region_of_interest_points], np.int32))
    cv2.imshow('Region of Interest', croppedImg)
    
    k = cv2.waitKey(25) & 0xFF
    if k == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()